In [39]:
import io
import re
import string
import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from ast import literal_eval

import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [2]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [3]:
path = '/content/drive/MyDrive/Graduate/Semester 1 (Fall 2024)/INFO-H423 Data Mining/Project/SNCB_IncidentClassifier/models/anomaly_detection/data'

In [26]:
df = pd.read_csv(f'{path}/AD_train_data.csv', index_col=0)
print(df.shape)
df.head()

(677, 10)


,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
238,4444185,"[1041, 1041, 1041, 1041, 1041, 1041, 1041, 104...","[4124, 4158, 4162, 4160, 4068, 2686, 3636, 365...","[-13815, -13653, -13639, -13637, -13213, -1320...",49.757405,5.535849,"[0.0, 94.8, 94.7, 94.4, 1.1, 0.0, 0.0, 0.0, 0....","[False, False, False, False, True, True, True,...","[True, False, False, False, False, False, Fals...",13
799,4469637,"[636, 660, 660, 636, 660, 660, 660, 660, 660, ...","[4066, 4068, 3636, 3658, 2956, 2956, 2956, 295...","[-14361, -14361, -14332, -14331, -14270, -1422...",51.186722,4.268448,"[0.3, 0.2, 0.0, 0.0, 29.1, 28.9, 27.8, 29.8, 7...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",14
6,4433287,"[578, 578, 578, 648, 648, 578, 578, 578, 648, ...","[2742, 4004, 4032, 4002, 4032, 2708, 4026, 402...","[-12414, -12414, -12414, -12413, -12413, -1241...",50.818278,3.255386,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",13
522,4457451,"[604, 604, 604, 604, 604, 604, 604, 604, 604, ...","[2956, 2956, 2956, 2956, 2956, 2956, 2956, 295...","[-14397, -14377, -14370, -14359, -14351, -1434...",51.086258,4.678054,"[37.8, 37.1, 44.1, 57.4, 65.3, 71.2, 74.8, 89....","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",14
476,4455435,"[1091, 510, 1091, 1091, 1091, 1091, 1091, 1091...","[3636, 3658, 4120, 2956, 2956, 2956, 2956, 295...","[-13340, -13339, -13320, -13240, -13207, -1319...",50.780656,4.649590,"[0.0, 0.0, 0.0, 29.0, 36.0, 38.4, 38.8, 37.9, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",2


In [27]:
cols_to_transform = ['vehicles_sequence', 'events_sequence', 'seconds_to_incident_sequence',
                     'train_kph_sequence', 'dj_ac_state_sequence', 'dj_dc_state_sequence']

for col in cols_to_transform:
    df[col] = df[col].apply(literal_eval)

In [28]:
events_list = df['events_sequence'].to_list()
events_flat = [j for sub in events_list for j in sub]
len(events_flat)

373983

In [30]:
unique_events = list(set(events_flat))
len(unique_events)

844

In [33]:
def process_seq(seqs, threshold):
    new_seqs = []
    for seq in seqs:
        if len(seq) > threshold:
            new_seq = seq[:threshold]
        else:
            new_seq = pad_sequences([seq], maxlen=threshold, padding='post')[0]
        new_seqs.append(new_seq)
    return new_seqs

In [34]:
embedding_dim = min(50, len(unique_events)//2 + 1)
max_seq_length = 750
vocab_size = len(unique_events)
num_classes = len(list(set(df['incident_type'].to_list())))
embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim)
embedding_dim

50

In [35]:
event_padded = process_seq(events_list, max_seq_length)
event_padded = np.array(event_padded)
event_padded.shape

(677, 750)

In [36]:
X = df['events_sequence'].to_list()
y = np.array(df['incident_type'].to_list())
y_encoder = LabelEncoder()
y_encoded = y_encoder.fit_transform(y)
y_encoded.shape

(677,)

In [37]:
X_text_og = np.array([' '.join(map(str, event_list)) for event_list in X])
X_text = np.array([' '.join(map(str, event_list)) for event_list in event_padded])
X_text.shape

(677,)

In [41]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=max_seq_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
vectorize_layer.adapt(X_text_og)

In [42]:
X_text_vectorized = vectorize_layer(X_text)

In [45]:
model = Sequential([
#   vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
#   Dense(16, activation='relu'),
  Dense(num_classes, activation='softmax')
])

opt = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [46]:
history = model.fit(x=X_text_vectorized, y=y_encoded, epochs=100)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2325 - loss: 2.1249
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3438 - loss: 1.8800
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3447 - loss: 1.8196
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3570 - loss: 1.7199
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3929 - loss: 1.6736
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4549 - loss: 1.6087
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4213 - loss: 1.6270
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4001 - loss: 1.6282
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4334 - loss: 1.5605
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4369 - loss: 1.5876
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4569 - loss: 1.4996
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - ac

In [47]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [48]:
weights.shape, len(vocab)

((844, 50), 844)

In [49]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()